1. Есть некоторое количество точек на плоскости, для которых известны взаимные расстояния, равные $\sqrt{E_i*E_j}r_{ij}$, $r_{ij}$ - евклидовое расстояние.  
2. Находим точки с минимальным   $\chi_{ij}$.  
3. Точки объединяются в одну, для них пересчитывается расстояние (энергетически взвешенно  $X_{new} = \frac{E_iX_i + E_jX_j}{E_i+E_j}$) и энергия.  
4. Переходим к следующей точке, пока расстояния между всеми точками меньше $\chi_{ij}$ или точки не закончатся.  

In [2]:
import pandas as pd
import numpy as np
import scipy
import matplotlib
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import math

from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import AgglomerativeClustering
import datetime
le = LabelEncoder()

from platform import python_version

print(python_version())

3.7.9


In [12]:
# Функция подсчета значений коэффициентов

def CoefPFI_analysis(df, num, cluster_column, hlabel_column):
    """
    Возвращает значения коэффициентов качества кластеризации в удобном для анализа формате.
    
    Входные значения:
    df - датафрейм. 
    num - номер семейства.
    cluster_column - название колонки с метками кластеров.
    hlabel_column - название колонки с метками высот.
    
    Возвращаемые значение:
    Печатает
    purity, splitting, integrity, efficiency, mean, compose
    
    (чистота, фрагментарность, целостность, эффективность, среднее, произведение средних).
    """
    
    P = 0
    E = 0
    DomH = [] #доминирующие высоты в кластерах
    N_c = len(df)
    
    if N_c==0:
        print('Empty Family')
        return
    
    labels = df[cluster_column]
    
    n_clusters_ = len(set(labels))
    
    for i in range(n_clusters_):
        
        value = df[df[cluster_column]==i][hlabel_column].value_counts()

        n_d = value[:1].values
        n_c = len(df[df[cluster_column]==i][hlabel_column])
        
        
        P_i = n_d[0]/n_c
        P = P + P_i
        
        x = value[:1].index[0]
        n_t = len(df[df[hlabel_column]==x])
        
        E_i = n_d[0]/n_t
        E = E + E_i
        DomH.append(x)
    
    purity = P/n_clusters_ #1
    
    splitting = len(set(DomH))/n_clusters_ #2
    
    integrity = len(set(DomH))/len(set(df[hlabel_column])) #3
 
    efficiency =  E/n_clusters_ #4
    
    print(f'P = {round(purity,3)}, S = {round(splitting,3)}, I = {round(integrity, 3)}, E = {round(efficiency, 3)}')
   
    
    result = [ num, round(purity, 3), round(splitting,3), round(integrity, 3), round(efficiency,3)] 
    mean = np.mean(result[1:])
    var = np.var(result[1:])
    compose = (purity+efficiency)*(integrity+splitting)/4
    
    result = result + [mean]
    result = result +[var]
    
    
    print(f'Mean: {round(mean, 3)}, compose: {round( compose , 3)}')
    
    return 

In [4]:
def Matrix_of_Distance(X, Y, E, label=2):
    """
    Возвращает матрицу рассстояний, посчитанную с использованием трёх разных метрик.
    label = 1 евклидово расстояние r_ij.
    label = 2 домножается на корень из энергий sqrt(e_i*e_j)*r_ij.
    label = 3 домножается на дробь (e_i*e_j)/(e_i+e_j)*r_ij.
    
    Входные значения:
    X - координата x, 
    Y - координата y,
    E - энергия, 
    label - маркер метрики расстояния.
    
    Возвращаемые значения:
    датафрейм матрицы взаимных расстояний.
    """
    # out = попарное евклидовое расстояние
    Z = np.array([[complex(X[i], Y[i]) for i in range(len(X))]])
    out = abs(Z.T-Z)
    
    if label==1:
        # евклидовое расстояние
        oute = 1
    elif label==2:
        # sqrt(e_i*e_j)
        oute = np.sqrt(np.outer(E, E))
    elif label==3:
        # e_i*e_j/(e_i+e_j)
        outmult = np.outer(E, E)
        outadd = np.add.outer(list(E), list(E))
        oute = outmult/outadd
        
    # Для треугольной формы    
    #oute = np.triu(oute)
    return pd.DataFrame(oute*out)

In [5]:
# Считываем данные модели
AllMc0 = pd.read_csv('datachanged/AllMc0C').drop(['Unnamed: 0'], axis = 1)

In [6]:
OneFamily = pd.DataFrame(AllMc0.loc[lambda AllMc0: AllMc0[' num_of_family'] == 1229, :]).copy()
OneFamily['Hlabel'] = le.fit_transform(OneFamily['H(J)'])

In [7]:
def clusters_change(clusters, val_to_change, val):
    """
    Заменяет значение в словаре, который содержит номера кластеров частиц семейства
    
    Входные значения:
    clusters - словарь кластеров 
    val_to_change - заменяемое значение (не ключ) 
    val - то значение, на которое нужно заменить
    
    Выходные значения:
    clusters - словарь кластеров
    """
    for key in clusters.keys():
        if clusters[key]==val_to_change:
            clusters[key] = val
    return clusters

In [8]:
def df_update(dfv, min_j):
    """
    Обновляет датафрейм, удаляя из него частицу.
    
    Входные значения:
    dfv - значения датафрейма (из метода values).
    min_j - номер удаляемой частицы.
    
    Выходные значения:
    dftmp - новый датафрейм.
    """
    
    dftmp = pd.DataFrame(dfv)
    dftmp = dftmp[dftmp[5]!=min_j].copy().reset_index(drop=True)
    check = len(dftmp)
    dftmp[5] = [i for i in range(check)]  
    return dftmp

In [10]:
def PamirAlgorithm(OneFamily, 
                   eps = 48, 
                   num_name = ' num_of_family',
                   j_name = ' j',
                   x_name = 'X(J)',
                   y_name = 'Y(J)',
                   e_name = 'E(J)',
                   alg_ind = 'j_new'):
    
    """
    Алгоритм Памира для метрики корень из произведения энергий на евклидовое расстояние sqrt(e_i*e_j)*r_ij.
    1. Есть некоторое количество точек на плоскости, для которых известны взаимные расстояния.  
    2. Находим точки с минимальным взаимным расстоянием.  
    3. Точки объединяются в одну энергетически взвешенно  X_new = (e_i*x_i + e_j*x_j)/(e_i+e_j) и энергия e_new = (e_i+e_j).  
    4. Переходим к следующей точке, пока расстояния между точками меньше 48 или точки не закончатся. 
    Значение 48 получено в процессе работы А.С.Борисова, можно изменить для экспериментов.
    
    Входные значения:
    OneFamily - датафрейм кластеризуемого семейства, 
    eps - параметр остановки, 
    num_name - имя колонки с номерами семейств,
    j_name - имя колонуи с номерами частиц,
    x_name - имя колонки с координатой x,
    y_name - имя колонки с координатой y,
    e_name - имя колонк с энергией,
    alg_ind - имя колонки для перенумерации частиц (могут быть пропуски в значениях).
    
    Возвращаемое значение:
    labels - колонка кастеров, порядок соответствует порядку частиц в колонке с номерами частиц j_name.
    """
    
    # создадим словарь кластеров, так как нумерация частиц может быть некорректной
    points = OneFamily[j_name].unique()
    clusters = {}
    for k in points:
        clusters[k]=-1
    k = 0

    OneFamily[alg_ind] = [i for i in range(len(OneFamily))] # добавляем вспомогательный индекс для выбрасывания частиц
    dfv = OneFamily[[num_name, j_name, x_name, y_name, e_name, alg_ind]].values
    dftmp = pd.DataFrame(dfv)
    check = len(dftmp)
    
    if (check==0):
        print('Empty Family')
        return
    
    minv = np.inf
    #print(check)

    while (check!=1) and ((minv < eps) or (minv==np.inf)):

        X = dftmp[2].values
        Y = dftmp[3].values
        E = dftmp[4].values

        distm = Matrix_of_Distance(X, Y, E)
        l = distm.shape[0]
        distm = distm.values

        min_i = 0
        min_j = 0
        minv = np.inf
        for j in range(l):
            for i in range(j+1, l):
                if distm[j][i]< minv:
                    min_i = i
                    min_j = j
                    minv = distm[j][i]

        #print(minv, min_i, min_j)


        if minv < eps:
            # если меньше, объединяем точки
            # первая точка
            ind_i = dfv[min_i][1]
            xi = dfv[min_i][2]
            yi = dfv[min_i][3]
            ei = dfv[min_i][4]

            # вторая точка
            ind_j = dfv[min_j][1]
            xj = dfv[min_j][2]
            yj = dfv[min_j][3]
            ej = dfv[min_j][4]
            
            #print(ind_i, ind_j)
            
            if (clusters[ind_j]!=-1) and (clusters[ind_i]==-1):
                #print('old', ind_j)
                #print('max_val=', max(clusters.values()))
                clusters[ind_i] = clusters[ind_j]
                #print(clusters.values())
            elif (clusters[ind_i]!=-1) and (clusters[ind_j]==-1):
                #print('old', ind_i)
                #print('max_val=', max(clusters.values()))
                clusters[ind_j] = clusters[ind_i]
                #print(clusters.values())
            elif (clusters[ind_j]==-1) or (clusters[ind_i]==-1):
                clusters[ind_i] = k
                clusters[ind_j] = k
                k= k + 1
                #print(clusters.values())
            else:
                #print('two old')

                if (clusters[ind_i]!=clusters[ind_j]):

                    val_to_change= clusters[ind_i]
                    #print('val_to_change=', clusters[ind_i])

                    max_val = max(clusters.values())
                    #print('max_val=', max_val)

                    #print('ind_i ', clusters[ind_i], 'ind_j ', clusters[ind_j])

                    # убираем совпавший кластер
                    clusters = clusters_change(clusters, val_to_change, clusters[ind_j])

                    # заменяем максимальный номер кластера на тот, что убираем
                    clusters = clusters_change(clusters, max_val, val_to_change)
                    k = k - 1

                # после обновления значений создаём новый датафрейм
                dftmp = pd.DataFrame(dfv)
                dftmp = dftmp[dftmp[5]!=min_j].copy().reset_index(drop=True)
                check = len(dftmp)
                dftmp[5] = [i for i in range(check)]
                dfv = dftmp.values
                # проверка длины
                check = len(dftmp)
                # пересчитываем индексы

                #print(clusters.values())
                continue

            dfv[min_i][1] = ind_i # j
            dfv[min_i][2] = (ei*xi+ej*xj)/(ei+ej) # x
            dfv[min_i][3] = (ei*yi+ej*yj)/(ei+ej)  # y
            dfv[min_i][4] = (ei+ej) # e
            dfv[min_i][5] = -1  # j_new
            #print(ind_i, ind_j)
            #print(min_i, min_j)

            # после обновления значений создаём новый датафрейм
            dftmp = pd.DataFrame(dfv)
            dftmp = dftmp[dftmp[5]!=min_j].copy().reset_index(drop=True)
            check = len(dftmp)
            dftmp[5] = [i for i in range(check)]
            dfv = dftmp.values
            # проверка длины
            check = len(dftmp) 
            #print('check =', check)
        else:
            #print('minv = ', minv)
            #print('k = ', k)
            for key in clusters.keys():
                if clusters[key]==-1:
                    clusters[key] = k
                    k = k + 1
    
                    
    labels = clusters.values()
    return labels

In [13]:
OneFamily['clust_pam']= PamirAlgorithm(OneFamily)
CoefPFI_analysis(OneFamily, ' num_of_fam', 'Hlabel', 'clust_pam')

Empty Family
Empty Family


In [999]:
chiri = np.percentile(OneFamily['ER'],50)
OneFamily['clust_pam']= PamirAlgorithm(OneFamily, chiri)
CoefPFI_analysis(OneFamily, ' num_of_fam', 'Hlabel', 'clust_pam')

P = 0.417, S = 1.0, I = 0.333, E = 1.0
Mean: 0.688, compose: 0.472


In [1002]:
X = Matrix_of_Distance(OneFamily['X(J)'].values, OneFamily['Y(J)'].values, OneFamily['E(J)'].values, 2)

In [1003]:
#linkage: average, complete, single.
thr = np.percentile(OneFamily, 50)
clustering = AgglomerativeClustering(n_clusters=None, affinity = 'precomputed', distance_threshold = thr, linkage = 'average').fit(X)
clustering.labels_

array([5, 0, 2, 7, 0, 2, 8, 4, 3, 6, 1], dtype=int64)

In [1004]:
OneFamily['clust_agl']= clustering.labels_

In [1005]:
CoefPFI_analysis(OneFamily, ' num_of_fam', 'Hlabel', 'clust_agl')

P = 0.361, S = 1.0, I = 0.222, E = 1.0
Mean: 0.646, compose: 0.416


In [1025]:
def clustering(data, name, exp = False):
    """
    Функция для проверки. 
    Кластеризует все семейства алгоритмом Памир.
    """
    nums =list(set(data[name].values))
    clustered_df = []
    
    print('start: ', datetime.datetime.now())
    
    for i in nums:
        if i%100 == 0:
            print(i, datetime.datetime.now())
        
        df = data[data[name]==i].copy()
        
        # eps = 48
        df['cluster_pam'] = PamirAlgorithm(df)
        
        clustered_df.append(df)
    
    clustered_df = pd.concat(clustered_df)

    return clustered_df

In [1026]:
# Старый модельный банк.
olddata = pd.read_csv('datachanged/AllMc0COld', sep = '\t')

In [1027]:
# Экспериментальные данные.
AllExp = pd.read_csv('datachanged/AllExpC')
AllExp = AllExp.drop(['Unnamed: 0'], axis = 1)

In [1028]:
clustered_df_old = clustering(olddata, ' num_of_family')

start:  2022-03-16 00:22:50.591990
10 2022-03-16 00:22:50.915161
20 2022-03-16 00:22:51.255247
30 2022-03-16 00:22:51.542447
40 2022-03-16 00:22:51.896500
50 2022-03-16 00:22:52.234598
60 2022-03-16 00:22:52.643502
80 2022-03-16 00:22:53.969957
90 2022-03-16 00:22:54.421748
100 2022-03-16 00:22:54.732924
110 2022-03-16 00:22:55.244546
120 2022-03-16 00:22:55.766153
130 2022-03-16 00:22:56.696665
140 2022-03-16 00:22:58.059021
150 2022-03-16 00:22:58.545718
160 2022-03-16 00:22:59.198013
170 2022-03-16 00:22:59.597949
180 2022-03-16 00:23:00.056716
190 2022-03-16 00:23:00.983202
200 2022-03-16 00:23:01.625520
210 2022-03-16 00:23:02.000517
220 2022-03-16 00:23:02.298706
230 2022-03-16 00:23:02.554003
240 2022-03-16 00:23:02.795358
250 2022-03-16 00:23:03.063665
260 2022-03-16 00:23:03.293026
270 2022-03-16 00:23:03.606188
280 2022-03-16 00:23:03.979052
290 2022-03-16 00:23:04.505648
300 2022-03-16 00:23:04.987324
310 2022-03-16 00:23:05.218703
320 2022-03-16 00:23:05.509925
330 2022-03-

In [1029]:
clustered_df_old

,num_of_family,j,X(J),Y(J),E(J),H(J),E0,A0,log_E0,R,ER,sum_energy,j_new,cluster_pam
0,1,1,-3.013189,-7.227429,4.611031,1255.2500,3366.712,1,3.527206,7.830392,36.106180,370.199507,0,3
1,1,2,-1.310529,-2.563248,12.047700,1545.0540,3366.712,1,3.527206,2.878841,34.683415,370.199507,1,3
2,1,3,-2.178048,-2.144561,5.401844,3044.5200,3366.712,1,3.527206,3.056638,16.511481,370.199507,2,3
3,1,4,-0.965465,-0.082880,5.325506,4715.5230,3366.712,1,3.527206,0.969016,5.160501,370.199507,3,2
4,1,5,-1.621645,0.079540,15.935060,3044.5200,3366.712,1,3.527206,1.623595,25.872076,370.199507,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27153,1298,6,-1.070832,-0.511355,5.577535,678.2601,25863.250,9,4.412683,1.186661,6.618646,226.805048,5,0
27154,1298,7,-0.093955,-0.075777,64.149700,678.2601,25863.250,9,4.412683,0.120705,7.743186,226.805048,6,0
27155,1298,8,-0.564117,-0.356699,27.180820,678.2601,25863.250,9,4.412683,0.667429,18.141268,226.805048,7,0
27156,1298,9,2.566935,2.136236,8.525503,45614.5600,25863.250,9,4.412683,3.339560,28.471427,226.805048,8,3


In [ ]:
eps = 48
num_name = ' num_of_family'
j_name = ' j'
x_name = 'X(J)'
y_name = 'Y(J)'
e_name = 'E(J)'
alg_ind = 'j_new'


# создадим словарь кластеров, так как нумерация частиц может быть некорректной
points = OneFamily[j_name].unique()
clusters = {}
for k in points:
    clusters[k]=-1
k = 0

OneFamily[alg_ind] = [i for i in range(len(OneFamily))] # добавляем вспомогательный индекс для выбрасывания частиц
dfv = OneFamily[[num_name, j_name, x_name, y_name, e_name, alg_ind]].values
dftmp = pd.DataFrame(dfv)
check = len(dftmp)

minv = np.inf
#print(check)

while (check!=1) and ((minv < eps) or (minv==np.inf)):
    
    X = dftmp[2].values
    Y = dftmp[3].values
    E = dftmp[4].values

    distm = Matrix_of_Distance(X, Y, E)
    l = distm.shape[0]
    distm = distm.values

    min_i = 0
    min_j = 0
    minv = np.inf
    for j in range(l):
        for i in range(j+1, l):
            if distm[j][i]< minv:
                min_i = i
                min_j = j
                minv = distm[j][i]
    #print(distm)
    print(minv, min_i, min_j)

    
    if minv < eps:
        # если меньше, объединяем точки
        # первая точка
        ind_i = dfv[min_i][1]
        xi = dfv[min_i][2]
        yi = dfv[min_i][3]
        ei = dfv[min_i][4]

        # вторая точка
        ind_j = dfv[min_j][1]
        xj = dfv[min_j][2]
        yj = dfv[min_j][3]
        ej = dfv[min_j][4]

        if (clusters[ind_j]!=-1) and (clusters[ind_i]==-1):
            print('old', ind_j)
            #print('max_val=', max(clusters.values()))
            clusters[ind_i] = clusters[ind_j]
            print(clusters.values())
        elif (clusters[ind_i]!=-1) and (clusters[ind_j]==-1):
            print('old', ind_i)
            #print('max_val=', max(clusters.values()))
            clusters[ind_j] = clusters[ind_i]
            print(clusters.values())
        elif (clusters[ind_j]==-1) or (clusters[ind_i]==-1):
            print('new')
            clusters[ind_i] = k
            clusters[ind_j] = k
            k= k + 1
            print(clusters.values())
        else:
            print('two old')

            if (clusters[ind_i]!=clusters[ind_j]):

                val_to_change= clusters[ind_i]
                #print('val_to_change=', clusters[ind_i])

                max_val = max(clusters.values())
                #print('max_val=', max_val)

                #print('ind_i ', clusters[ind_i], 'ind_j ', clusters[ind_j])

                # убираем совпавший кластер
                clusters = clusters_change(clusters, val_to_change, clusters[ind_j])

                # заменяем максимальный номер кластера на тот, что убираем
                clusters = clusters_change(clusters, max_val, val_to_change)
                k = k - 1

            # после обновления значений создаём новый датафрейм
            dftmp = df_update(dfv, min_j)
            dfv = dftmp.values
            # проверка длины
            check = len(dfv)
            # пересчитываем индексы
            
            print(clusters.values())
            continue

        dfv[min_i][1] = ind_i # j
        dfv[min_i][2] = (ei*xi+ej*xj)/(ei+ej) # x
        dfv[min_i][3] = (ei*yi+ej*yj)/(ei+ej)  # y
        dfv[min_i][4] = (ei+ej) # e
        dfv[min_i][5] = -1  # j_new
        #print(ind_i, ind_j)
        #print(min_i, min_j)

        # после обновления значений создаём новый датафрейм
        dftmp = df_update(dfv, min_j)
        dfv = dftmp.values
        # проверка длины
        check = len(dfv)     
    else:
        print('minv = ', minv)
        #print('k = ', k)
        for key in clusters.keys():
            if clusters[key]==-1:
                clusters[key] = k
                k = k + 1